# Introduction

In this lesson we will learn a little about how to use Python for cleaning input data, including using regular expressions, as well as the basic idea behind probabilistic record linkage. These two topics fit together naturally, because data cleaning can have a signficant impact on the success of record linkage. Being able to compare fields that were normalized the same way will give better results.

We will use two data sets for the exercises in this chapter. The first will be a list of NIH projects and researchers pulled from the class database. The second is a list of university employees that was scraped from public web sites.

# Data Definition

Before we begin the task of record linkage, it's important that we understand the variables in our data. In this workbook, we will take a cursory look at some of the values in our data and compute some simple statistics to ensure that the content makes sense. 

Begin by loading the two data sets into pandas data frames. The first file uses tabs as field separators, and we specify this in the `read_csv` method. The second data set contains a zip code field that pandas will try to interpret as a number by default. To prevent this, we explicity set the numpy dtype (data type) that we want to use for that column. After we load the two data sets, we all the `head` method on the first data set to examine the first few reords.

In [58]:
import numpy as np
import pandas as pd

# specify that the file uses tabs as separators
uc = pd.read_csv("ucpay2011.csv", sep="\t")

# the zip code field should be read as a character string
nsf = pd.read_csv("nsf_awards_2010-2012.csv", dtype={"ZipCode": np.str})

# get the first few records from the UC employee file.
uc.head()

,ID,year,campus,name,title,gross,base,overtime,extra,exclude
0,1751971,2011,BERKELEY,***********,TUTOR - NON-GSHIP,0.49,0,0,0,0
1,1758984,2011,BERKELEY,***********,TUTOR - NON-GSHIP,0.49,0,0,0,0
2,1821585,2011,IRVINE,***********,TUTOR - NON-GSHIP,0.51,0,0,0,0
3,1966846,2011,SAN FRANCISCO,"MACKEWICZ , CARL E",STAFF RESEARCH ASSOC III,0.62,0,0,0,0
4,1758947,2011,BERKELEY,***********,READER - NON-GSHIP,0.73,0,0,0,0


Here are some initial thoughts about the data:
* The ID field looks like a unique identifier that is specific to the data set. If we thought that we were going to use this identifier to link to other data from the same source, then we might take a loser look to see if the values should be interpreted as numbers or character strings. Ideally, this information would appear in the data documentation.
* We should check to make sure that the year field contains valid values.
* The campus and title fields look like they should be interpreted as finite categorical variables.
* The name field appears to contain some redacted values, probably due to a privacy agreement. We will want to link the subset of valid name fields.

Let's perform some quick summaries of the fields in the data. We'll ignore the numeric columns for now, because we won't be using them for linkage, but a thorough data definition process would ensure that these columns are valid as well.

In [59]:
# print the distinct values in the year, campus, and title columns
print("Distinct years = ", uc["year"].unique())
print("Distinct campuses = ", uc["campus"].unique())
print("Distinct titles = ", uc["title"].unique())

# There are too many titles to display, so let's get the count
print("Number of distinct titles = ", len(uc["title"].unique()))

# Print the total number of rows and the number of rows with valid name values
print("Total rows = ", len(uc))
name = uc["name"]
print("Rows with valid names = ", len(name[name != "***********"]))


('Distinct years = ', array([2011], dtype=int64))
('Distinct campuses = ', array(['BERKELEY', 'IRVINE', 'SAN FRANCISCO', 'LOS ANGELES', 'DANR',
       'SANTA BARBARA', 'SANTA CRUZ', 'RIVERSIDE', 'DAVIS', 'MERCED',
       'SAN DIEGO', 'UCOP'], dtype=object))
('Distinct titles = ', array(['TUTOR - NON-GSHIP', 'STAFF RESEARCH ASSOC III',
       'READER - NON-GSHIP', ..., 'ATHLETICS MANAGER 4',
       'CHIEF EXEC OFFICER - MED CENTR', 'TREASURER OF THE REGENTS'], dtype=object))
('Number of distinct titles = ', 2626)
('Total rows = ', 259043)
('Rows with valid names = ', 163429)


Next we'll take a look at the second data set.

In [60]:
nsf.head()

,AwardId,FirstName,LastName,StartDate,EndDate,AwardTitle,AwardEffectiveDate,AwardExpirationDate,Name,CityName,ZipCode,PhoneNumber,StreetAddress,CountryName,StateName,StateCode
0,415302,Jeffrey,Kuhn,1/15/2010,NaN,Advanced Technology Solar Telescope (ATST) Con...,1/1/2010,9/30/2015,Association of Universities for Research in As...,Washington,200053929,2024832101,"1212 New York Avenue, N.W.,",United States,District of Columbia,DC
1,415302,Robert,Rosner,1/15/2010,NaN,Advanced Technology Solar Telescope (ATST) Con...,1/1/2010,9/30/2015,Association of Universities for Research in As...,Washington,200053929,2024832101,"1212 New York Avenue, N.W.,",United States,District of Columbia,DC
2,415302,Philip,Goode,1/15/2010,NaN,Advanced Technology Solar Telescope (ATST) Con...,1/1/2010,9/30/2015,Association of Universities for Research in As...,Washington,200053929,2024832101,"1212 New York Avenue, N.W.,",United States,District of Columbia,DC
3,415302,Thomas,Rimmele,3/15/2012,NaN,Advanced Technology Solar Telescope (ATST) Con...,1/1/2010,9/30/2015,Association of Universities for Research in As...,Washington,200053929,2024832101,"1212 New York Avenue, N.W.,",United States,District of Columbia,DC
4,415302,Stephen,Keil,1/15/2010,3/15/2012,Advanced Technology Solar Telescope (ATST) Con...,1/1/2010,9/30/2015,Association of Universities for Research in As...,Washington,200053929,2024832101,"1212 New York Avenue, N.W.,",United States,District of Columbia,DC


Observations:
* There are separate fields for first name and last name
* The data include researchers from universities throughout the US, not just in the UC system
* For UC schools we should be able to create a field corresponding to the campus field in the UC data, but this will require some recoding.

In the next two sections we will address some of the data issues that we've identied. Before moving on, we will update the `uc` data set so that it only contains records with valid name fields.

In [61]:
uc_name = uc[name != "***********"]
uc_name.head()

,ID,year,campus,name,title,gross,base,overtime,extra,exclude
3,1966846,2011,SAN FRANCISCO,"MACKEWICZ , CARL E",STAFF RESEARCH ASSOC III,0.62,0.00,0,0,0
6,1870390,2011,LOS ANGELES,"ESCUJURI , ERIC JOSEPH","TECHNICIAN, SCENE, SR",1.14,0.00,0,0,0
7,1771936,2011,BERKELEY,"JUNG , WOO YONG",POSTDOC-EMPLOYEE,1.28,1.28,0,0,0
8,1892122,2011,LOS ANGELES,"SHAPIRO , JORDAN ISAAC","TECHNICIAN, SCENE",1.33,0.00,0,0,0
12,1988359,2011,SANTA BARBARA,"CUTLER , CHARLES IAN",LABORATORY ASST I,1.55,0.00,0,0,0


# Parsing

Above, we noted that we need to parse the name field in the UC data into first, middle, and last name fields. First, let's see what we can do with the built-in `split` method. By default, the `split` method returns a list of strings obtained by splitting the original string on spaces:

In [62]:
print("Lorem Ipsum".split())
print("carrot cake".split())
print("Bogart, Humphrey".split())

['Lorem', 'Ipsum']
['carrot', 'cake']
['Bogart,', 'Humphrey']


By default, `split` treats whitespae as delimiting characters, and multiple consecutive whitespace characters are treated like a single delimiter. We can also pass an argument to `split` to be the delimiter. If we set the delimiter explicitly, then multiple delimiters are not combined.

In [63]:
print("Hi,       Mom!".split())
print("Hi,       Mom".split(","))

# in the following example, the result will include the empty strings between the first and second commas
# and the second and third commas
print("Hi,,,Mom!".split(","))

['Hi,', 'Mom!']
['Hi', '       Mom']
['Hi', '', '', 'Mom!']


We can map the `split` function over the values in the UC employee `name` column.

In [64]:
uc_name["name"].apply(str.split).head()

3         [MACKEWICZ, ,, CARL, E]
6     [ESCUJURI, ,, ERIC, JOSEPH]
7            [JUNG, ,, WOO, YONG]
8     [SHAPIRO, ,, JORDAN, ISAAC]
12      [CUTLER, ,, CHARLES, IAN]
Name: name, dtype: object

This is pretty good! Fortunately for us, the UC data includes a space between the last name and the comma separating it from the first name. This is unusual, but it means that the built-in `split` method is sufficient. We can define functions `getlastname` and `getfirstname` that will call the `split` method and extrac the appropriate values from the resulting list. We won't worry about middle names, because the NSF data doesn't include them.

It turns out that all name fields in the `uc_name` data frame contain more than one word, even after filtering out the redacted names. Our `getfirstname` function will have to perform a check to make sure that there is a first name to get.

Note that the following code displays a warning which we can safely ignore.

In [65]:
def getfirstname(name):
    parts = name.split()
    
    if len(parts) >= 3:
        return parts[2]
    else:
        return None

def getlastname(name):
    parts = name.split()
    return parts[0]

uc_name["firstname"] = uc_name["name"].apply(getfirstname)
uc_name["lastname"] = uc_name["name"].apply(getlastname)

# Let's see the result
uc_name[["name", "firstname", "lastname"]].head(15)

C:\Python27\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python27\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,firstname,lastname
3,"MACKEWICZ , CARL E",CARL,MACKEWICZ
6,"ESCUJURI , ERIC JOSEPH",ERIC,ESCUJURI
7,"JUNG , WOO YONG",WOO,JUNG
8,"SHAPIRO , JORDAN ISAAC",JORDAN,SHAPIRO
12,"CUTLER , CHARLES IAN",CHARLES,CUTLER
15,"ANDERSON , MARK CALVIN",MARK,ANDERSON
25,"PATEL , DEV KAPIL",DEV,PATEL
26,"HILDER , JAMIE L",JAMIE,HILDER
46,"WU , YALEI",YALEI,WU
73,"VALERIO , STEVEN GERARD,JR",STEVEN,VALERIO


I intentionally showed 15 rows instead of the default 10 so that we could see an example of our simple approach failing. The second to last row in the result contains a last name that comprises several words. Our rule, which extracted the first word to be the "last name" does not give the correct result.

There is a more subtle problem as well. It's likely that the third name, WOO YONG, should not be split into a first name and middle name component. That is, WOO YONG may in fact be the first name of the referenced individual. This format is common for names from some countries, including China and Korea.

Let's redefine the `getfirstname` and `getlastname` functions so that the last name consists of everything before the comma and the first name consists of the first word after the comma. This doesn't solve our problem with Chinese and Korean names, but it's a good start.

In [66]:
def getfirstname(name):
    parts = name.split(",")
    
    if len(parts) > 1:
        # The name contains a comma. Take the part after the comma, split on whitespace,
        # and grab the first word
        return parts[1].split()[0]
    else:
        return None

def getlastname(name):
    parts = name.split(",")
    
    # Return the first part (everything up to the first comma)
    # call the `strip` method to remove the space between the last name and the comma
    return parts[0].strip()

uc_name["firstname"] = uc_name["name"].apply(getfirstname)
uc_name["lastname"] = uc_name["name"].apply(getlastname)

# Let's see the result
uc_name[["name", "firstname", "lastname"]].head(15)

C:\Python27\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python27\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,firstname,lastname
3,"MACKEWICZ , CARL E",CARL,MACKEWICZ
6,"ESCUJURI , ERIC JOSEPH",ERIC,ESCUJURI
7,"JUNG , WOO YONG",WOO,JUNG
8,"SHAPIRO , JORDAN ISAAC",JORDAN,SHAPIRO
12,"CUTLER , CHARLES IAN",CHARLES,CUTLER
15,"ANDERSON , MARK CALVIN",MARK,ANDERSON
25,"PATEL , DEV KAPIL",DEV,PATEL
26,"HILDER , JAMIE L",JAMIE,HILDER
46,"WU , YALEI",YALEI,WU
73,"VALERIO , STEVEN GERARD,JR",STEVEN,VALERIO


# Regular Expressions

We were able to solve our parsing problem using the `split` method. Sometimes we need a more heavy duty way to search and extract text. In this section we introduce the basics of regular expressions, because they are a common approach to text parsing and a powerful one, too.

Regular expressions are a mini-language for searching text. We have already noted that regular expressions are *powerful*, but they can also be *complex*. 

> Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems. 

> &mdash; Jamie Zawinski

In the previous section we were trying to extract first and last names from the UC name field:

In [67]:
uc_name["name"].head()

3         MACKEWICZ , CARL E
6     ESCUJURI , ERIC JOSEPH
7            JUNG , WOO YONG
8     SHAPIRO , JORDAN ISAAC
12      CUTLER , CHARLES IAN
Name: name, dtype: object

We used basic python functions to try to split the names into first and last name components. In this section we'll use regular expressions to accomplish this task.

In [68]:
import re

In words, this is how I would explain where to find the last name component in the author field: "starting from the beginning of the line, take all the characters until you see a comma." We can build a regular expression that captures this idea from the following components:
* `^` Matches beginning of the line
* `.` Matches any character
* `+` A modifier that means "match one or more of the preceding expression"
The comma is not a special character in regular expressions, so it will match itself.

In [69]:
m = re.search(r"^.+,", "Su, Yu-Wen")
m

Two things:
* We used "raw" string syntax `r""` to prevent python from escaping any special characters that might appear in our regular expression.
* If the regular expression matches something in the searched string, then `m` holds a python match-data object. Otherwise, it will be `None`.

To see the matched substring, use the `group` function.

In [70]:
m.group()

'Su,'

This is almost what we want, but it would be nice if we weren't also capturing the comma. The problem is that we need the comma in our regular expression in order to know where the last name ends. The solution is to use regular expression groups by putting parentheses around parts of the regular expression that we want to refer to later.
* `(...)` Creates a group that can be referred to later.

In [71]:
m = re.search(r"^(.*),", "Franklin, Benjamin")

print(m.group())
print(m.group(1))

Franklin,
Franklin


In a match-data object `m`, `m.group(1)` will show you what was matched by the expression between the first set of parentheses,  `m.group(2)` will show you what was matched by the expression between the second set of parentheses, and so on. `m.group()` is a synonym for `m.group(0)` and it always refers to what was matched by the entire regular expression.

Now let's write a regular expression that will match the first name of the employee (that is, the first word after the comma). In addition to the bits of regular expressions we saw above, here are some other useful character classes:
* `\w` Matches a single alphanumeric character
* `\W` Matches anything that is not an alphanumeric character
* `\s` Matches a single whitespace character
* `\S` Matches anything that is not a whitespace character.

In [72]:
regex = r",\s*(\w+)"
m = re.search(regex, "Eisenhower, Dwight David")
m.group(1)

'Dwight'

Regular expressions are a powerful tool and we're barely scratching the surface. 

## References
* Python documentation: https://docs.python.org/2/library/re.html#regular-expression-syntax
* Online regular expression tester (good for learning): http://regex101.com/

# String Comparators

In this section we will demonstrate different string comparator algorithms which are provided by the [jellyfish](https://github.com/sunlightlabs/jellyfish) package by writing a function that takes a name as an argument and returns the names in the NSF data that are most similar to the argument. We will start by importing the package and creating a `set` of unique first names from the NSF data. The `FistName` field is missing some values which are represented as NaN in the data frame. To prevent errors later on, we only include valid character strings (which have type `str`) in our list of unique names.

In [73]:
import jellyfish as jf

uniq_first_names = set(name for name in nsf["FirstName"] if type(name) == str)

Next, we will define our function. It should:
* Take a name (i.e., a string) as an argument
* Optionally, take a number `n` which determines the size of the output
* Compare the name to each name in `uniq_first_names` using the Levenshtein distance string comparator
* Return a list of the `n` names in `uniq_first_names` that are closest to the argument

Here's one implementation. Note that in the comparison we capitalize both names being compared, so that letter case doesn't affect the final distance.

In [74]:
def closest_names(name, n=10):
    # First create a list of tuples (other_name, distance), where
    # other_name is taken from uniq_first_names
    distances = [(other_name, jf.levenshtein_distance(unicode(name.upper()), unicode(other_name.upper()))) 
                 for other_name in uniq_first_names]
    
    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1])[:n]

Let's try it out on some names.

In [75]:
print(closest_names("Jennifer"))
print(closest_names("Sonya"))
print(closest_names("Wai Tong"))

[('Jennifer', 0), ('Jenifer', 1), ('Jennie', 2), ('Jeanine', 3), ('Jeannine', 3), ('Jeannie', 3), ('Jennifer A.', 3), ('Annie', 4), ('Lonnie', 4), ('Werner', 4)]
[('Sonya', 0), ('Sonia', 1), ('Sona', 1), ('Sofya', 1), ('Sonja', 1), ('Tonya', 1), ('Sofia', 2), ('Sanja', 2), ('Roya', 2), ('Tony', 2)]
[('Hailong', 3), ('Jae Hong', 3), ('Wai Tak', 3), ('Airong', 3), ('Aidong', 3), ('Yitong', 3), ('Weizhong', 3), ('Weidong', 3), ('Weiyong', 3), ('Hairong', 3)]


Recall that Levenshtein distance is a kind of edit distance. Edit distances count the number of edit operations needed to change one word to another, and different edit distances count different edit operations as valid. In the case of Levenshtein distance, the valid edit operations are inserting a letter, deleting a letter, or changing a letter. 

It would be interesting to compare this output to the output from other string comparators included in the jellyfish package:
* Levenshtein distance: edit distance where the valid operations are inserting a letter, deleting a letter, or changing a letter
* Levenshtein-Damerau distance: edit distance which includes the same operations as Levenshtein distance but also allows transposing two adjacent letters. This can be useful for finding words with typos.
* Jaro-Winkler distance: a fast-to-compute string distance based on common letters between two words

Let's update our `closest_names` function so that we can specify the string comparator we want to use. Note that for edit distance smaller numbers indicate closer strings, but for Jaro-Winkler distance larger values indicate closer strings. We will have to add an option to our new function that lets us specify the sort order that determines which strings are close.

In [76]:
def closest_names_2(string_comparator, name, reverse_sort=False, n=10):
    # First create a list of tuples (other_name, distance), where
    # other_name is taken from uniq_first_names
    distances = [(other_name, string_comparator(unicode(name.upper()), unicode(other_name.upper())))
                 for other_name in uniq_first_names]
    
    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1], reverse=reverse_sort)[:n]

In [77]:
# Try it!
print(closest_names_2(jf.damerau_levenshtein_distance, "William"))
print(closest_names_2(jf.jaro_winkler, "Wiliam", reverse_sort=True))

[('william', 0), ('William', 0), ('Williams', 1), ('Willa', 2), ('Fillia', 2), ('William J', 2), ('Gillian', 2), ('Lillian', 2), ('J.William', 2), ('Willie', 2)]
[('william', 0.9666666666666667), ('William', 0.9666666666666667), ('Williams', 0.9416666666666667), ('William J', 0.9222222222222222), ('J.William', 0.8888888888888888), ('Ilia', 0.8888888888888888), ('Willa', 0.8755555555555556), ('William Rogers', 0.8666666666666668), ('H. William', 0.8666666666666667), ('Willie', 0.8444444444444443)]


# Fellegi-Sunter Record Linkage

In this section we will "manually" perform the steps in Fellegi-Sunter record linkage. Our goal is to illustrate the Fellegi-Sunter algorithm by breaking it into bitesize pieces. 

In our example we will compare first names and last names using Levenshtein distance. In the Fellegi-Sunter algorithm, the result of a field comparison is assumed to follow a multinomial distribution. That means it can only take on finitely many values. Therefore we will define a function that compares to strings and returns the value 2, 1, or 0 to indicate an exact match, a nearly exact match, or anything else.

In [78]:
def fuzzy_string_comparator(s1, s2):
    if type(s1) != str or type(s2) != str:
        return 0
    
    dist = jf.jaro_winkler(unicode(s1.upper()), unicode(s2.upper()))
    if dist >= 0.92:
        return 2
    if dist >= 0.85:
        return 1
    else:
        return 0
    
print(fuzzy_string_comparator("joshua", "joshua"))
print(fuzzy_string_comparator("joshua", "joshau"))
print(fuzzy_string_comparator("joshua", "todd"))

2
2
0


The above function compares *fields* in a record. Next, we define a function that compares *records*. My record comparison function assumes that records will have the form of a tuple: (identifier, first name, last name). It returns a length 2 tuple that gives the result of applying our fuzzy string comparator to the first name and to the last name.

In [79]:
def comparison_vector(rec1, rec2):
    """rec1 and rec2 have the form (id, first name, last name)"""
    m = fuzzy_string_comparator(rec1[1], rec2[1])
    n = fuzzy_string_comparator(rec1[2], rec2[2])
    return (m, n)

print(comparison_vector((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(comparison_vector((3, "joshua", "tokle"), (4, "josue", "tolke")))

(2, 0)
(1, 2)


Next, we need to define m-weights and u-weights. An m-weight is the probability of seeing a particular field comparison outcome given that we are comparing two records that represent the same individual. A u-weight is the probability of seeing a particular field outcome given that we are comparing two records that do *not* represent the same individual. For example, if two records represent the same person, the first and last names should match with high probability (the m-weights for the comparison value 2 should be large). On the other hand, suppose we had month of birth in our data set. The probability that two random individuals will agree on month of birth is about 1/12, so we would assign a u-weight of about 1/12 to this outcome.

I'm going to make up some m-weights and u-weights here. In practice you might try to fit these to data or tweak them after seeing preliminary output. I don't claim that these are particularly good.

In [80]:
# The outer list is length 2, corresponding to the number of fields we're comparing.
# Each inner list has length 3, because fuzzy string comparator returns 3 possible values.

m_weights = [[0.01, 0.14, 0.85], # m-weights corresponding to first name
             [0.01, 0.09, 0.90]] # m-weights corresponding to last name

u_weights = [[0.88, 0.10, 0.02], # u-weights corresponding to first name
             [0.91, 0.08, 0.01]] # u-weights corresponding to last name

Now we define a function that compares two records and returns the match score.

In [81]:
import math

def match_score(rec1, rec2):
    m, n = comparison_vector(rec1, rec2)
    
    # what's the log-probability of seeing this comparison vector if the records are a match?
    log_pr_given_match = math.log(m_weights[0][m]) + math.log(m_weights[1][n])
    
    # what's the log-probability of seeing this comparison vector if the records are a nonmatch?
    log_pr_given_nonmatch = math.log(u_weights[0][m]) + math.log(u_weights[1][n])
    
    # return the log-likelihood-ratio
    return log_pr_given_match - log_pr_given_nonmatch

print(match_score((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(match_score((1, "joshua", "tokle"), (4, "joshu", "tolke")))
print(match_score((1, "joshua", "tokle"), (7, "christina", "jones")))

-0.761355430586
8.24931374626
-8.988196321


Finally, we can try to link UC employees to NSF researchers. We will apply **blocking** to reduce the number of comparisons we perform. In particular, I am only going to compare graduate students whose last name begins with the letter "H". I will also limit the NSF researchers to those located in California.

In [82]:
startswith_h = uc_name["lastname"].apply(lambda s: s[0] == "H")
uc_h = uc_name[startswith_h]

startswith_h = nsf["LastName"].apply(lambda s: type(s) == str and s[0].upper() == "H")
nsf_h = nsf[startswith_h]

*The following chunk is time-consuming.*

In [ ]:
# This cell takes a minute to execute

potential_matches = []

for _, uc_row in uc_h.iterrows():
    rec1 = (uc_row["ID"], uc_row["firstname"], uc_row["lastname"])
    for nsf_ix, nsf_row in nsf_h.iterrows():
        rec2 = (nsf_ix, nsf_row["FirstName"], nsf_row["LastName"])
        score = match_score(rec1, rec2)
        if score >= 0.5:
            potential_matches.append((score, rec1, rec2))
            
# Sort the output so the best matches appear at the top
potential_matches = sorted(potential_matches, key=lambda x: x[0], reverse=True)
potential_matches

# How did we do?